In [3]:
!pip3 install numpy pandas matplotlib sparqlwrapper

Collecte de données

In [7]:
import requests
import glob
import shutil
import os
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?tour ?tourLabel ?image {
  ?tour wdt:P31 wd:Q12518;
          wdt:P18 ?image.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["tourLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["tour", "image"])
dataframe = dataframe.astype(
    dtype={"tour": "<U200", "image": "<U200"}
)



def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open(os.path.join("images", os.path.basename(url)), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

dataframe.image.apply(download_image)

0     200
1     200
2     200
3     200
4     200
     ... 
95    200
96    200
97    200
98    200
99    200
Name: image, Length: 100, dtype: int64

Métadonnées des images

In [51]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS
import numpy as np

# Effacer le contenu du fichier au début du programme
open("images/image_tags.json", 'w').close()

def get_exif_data(img):
    exif_data = img.getexif()
    exif = {}
    if exif_data:
        for tag, value in exif_data.items():
            if tag in TAGS:
                if isinstance(value, bytes):
                    value = value.decode(errors='ignore')
                elif isinstance(value, (Fraction, int, float)):
                    value = str(value)
                elif isinstance(value, tuple):
                    value = tuple(str(v) for v in value)
                else:
                    value = str(value)
                exif[TAGS[tag]] = value
    return exif

def get_image_metadata(image_path):
    img = Image.open(image_path)
    img = img.convert("RGB")  # Ensure image is in RGB mode

    exif_data = get_exif_data(img)
    exif_data.pop('MakerNote', None)
    img = img.resize((100, 100))  # Redimensionner pour accélérer le traitement
    img_array = np.array(img)
    img_array = img_array.reshape((img_array.shape[0] * img_array.shape[1], 3))
    
    kmeans = KMeans(n_clusters=5, n_init=10)
    kmeans.fit(img_array)
    
    # Obtenir les couleurs des clusters
    dominant_colors = kmeans.cluster_centers_
    metadata = {
        "filename": os.path.basename(image_path),
        "format": img.format,
        "size": img.size,
        "mode": img.mode,
        "orientation": "landscape" if img.width > img.height else "portrait",
        "exif": exif_data,
        "dominant_colors": [dominant_colors[i].tolist() for i in np.argsort(dominant_colors.sum(axis=1))],
    }
    return metadata

def save_metadata_to_json(metadata, output_file="images/image_tags.json"):
    try:
        with open(output_file, 'r') as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []

    data.append(metadata)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

image_files = [f"images/{f}" for f in os.listdir('images') if f.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'))]
for image_path in image_files:
    metadata = get_image_metadata(image_path)
    save_metadata_to_json(metadata)
    print(f"Métadonnées de l'image {image_path} enregistrées.")

Métadonnées de l'image images/Grosshaufmt1p.jpg enregistrées.
Métadonnées de l'image images/2010-03-23%20Broitzem-TV-Turm.JPG enregistrées.
Métadonnées de l'image images/Birk%20%28Lohmar%29%20Fernmeldeturm.jpg enregistrées.
Métadonnées de l'image images/Merkendorf%2C%20Unteres%20Tor-001.jpg enregistrées.
Métadonnées de l'image images/London%20Lanthorn%20Tower%2008.03.2013%2013-39-42.JPG enregistrées.
Métadonnées de l'image images/Mannesmann-tower-hannover-messe.jpg enregistrées.
Métadonnées de l'image images/Trier%20Frankenturm%20BW%201.JPG enregistrées.
Métadonnées de l'image images/P1000625%20Paris%20IV%20Tour%20Saint-Jacques%20resultwk.JPG enregistrées.
Métadonnées de l'image images/Bautzen%20M%C3%BChlbastei%202.JPG enregistrées.
Métadonnées de l'image images/Ln0pZand2.jpg enregistrées.
Métadonnées de l'image images/D%C3%BClmen%2C%20Nonnenturm%20--%202012%20--%204152.jpg enregistrées.
Métadonnées de l'image images/2008-05-16%20Handwerkerhof%20N%C3%BCrnberg%20Frauentor.jpg enregistré

In [14]:
import json
import os
import random

# Create the users directory if it doesn't exist
if not os.path.exists('users'):
    os.makedirs('users')

# Get metadata for all images
with open('images/image_tags.json', 'r') as f:
    metadata = json.load(f)

# Get all image filenames
image_filenames = [img['filename'] for img in metadata]

# Create a copy of image_filenames to draw from
available_images = image_filenames.copy()

# Dictionary to hold user data
users_data = {}

# For each user (1 to 5)
for i in range(5):
    # Randomly select 5 unique images for this user
    user_images = random.sample(available_images, 5)
    
    # Remove selected images from available pool
    for img in user_images:
        available_images.remove(img)
    
    # Add user data to dictionary
    users_data[f'user{i+1}'] = {
        'images': user_images,
        # You can add more user-specific information here
        'additional_info': {}
    }

# Save to a single JSON file
with open('users/users.json', 'w', encoding='utf-8') as f:
    json.dump(users_data, f, ensure_ascii=False, indent=2)

print("User file created with 5 unique images each")


User file created with 5 unique images each


In [74]:
import json
import numpy as np
from sklearn.cluster import KMeans

# Charger les données des utilisateurs
with open('users/users.json', 'r') as f:
    users_data = json.load(f)

# Charger les métadonnées des images
with open('images/image_tags.json', 'r') as f:
    image_metadata = json.load(f)

for i in range(1, 6):
    # Récupérer les images de l'utilisateur
    user_images = users_data[f'user{i}']['images']

    # Récupérer les couleurs dominantes pour chaque image préférée de l'utilisateur
    dominant_colors = []
    for image_filename in user_images:
        for img_data in image_metadata:
            if img_data['filename'] == image_filename:
                dominant_colors.extend(img_data['dominant_colors'])
                break

    # Convertir les couleurs dominantes en tableau numpy
    dominant_colors = np.array(dominant_colors)

    # Utiliser KMeans pour trouver la couleur la plus présente parmi les couleurs dominantes
    kmeans = KMeans(n_clusters=1, n_init=10)
    kmeans.fit(dominant_colors)
    most_present_color = kmeans.cluster_centers_[0]

    # Ajouter la couleur dominante aux informations supplémentaires de l'utilisateur
    users_data[f'user{i}']['additional_info']['dominant_color'] = most_present_color.tolist()

# Sauvegarder les données mises à jour dans le fichier users.json
with open('users/users.json', 'w', encoding='utf-8') as f:
    json.dump(users_data, f, ensure_ascii=False, indent=2)

print("Couleurs dominantes ajoutées aux informations des utilisateurs.")


Couleurs dominantes ajoutées aux informations des utilisateurs.
